# Import and get the version

In [50]:
import mlx
import mlx.core as mx


# Create an scalar

In [51]:
def print_scalar_info(scalar):
    """
    Print detailed information about the given scalar tensor.

    Parameters:
      scalar (mlx.core.array): A tensor containing a single scalar value.
    """
    print(f"Scalar:\n{scalar}\n")
    print(f"Scalar type:\n{type(scalar)}\n")
    print(f"Scalar dimension:\n{scalar.ndim}\n")
    
    # Get the Python number within the tensor (only works with one-element tensors)
    item = scalar.item()
    print(f"Scalar item:\n{item}\n")


scalar = mx.array(7)
print_scalar_info(scalar)


Scalar:
7

Scalar type:
<class 'mlx.core.array'>

Scalar dimension:
0

Scalar item:
7



# Create a vector, matrix and tensor

In [52]:
def print_tensor_info(tensor):
    """
    Print detailed information about the given tensor of at least 2 elements.

    Parameters:
      Tensor (mlx.core.array): A tensor containing a single scalar value.
    """
    print(f"Tensor:\n{tensor}\n")
    print(f"Tensor type:\n{type(tensor)}\n")
    print(f"Tensor dimension:\n{tensor.ndim}\n")
    print(f"Tensor shape:\n{tensor.shape}\n")

vector = mx.array([7, 7])  
print_tensor_info(vector)
    

Tensor:
array([7, 7], dtype=int32)

Tensor type:
<class 'mlx.core.array'>

Tensor dimension:
1

Tensor shape:
(2,)



In [53]:
matrix = mx.array([[7, 8],[9, 10]])
print_tensor_info(matrix)

Tensor:
array([[7, 8],
       [9, 10]], dtype=int32)

Tensor type:
<class 'mlx.core.array'>

Tensor dimension:
2

Tensor shape:
(2, 2)



In [54]:
tensor_3d = mx.array([[[1, 2], [3, 4]]])  
print_tensor_info(tensor_3d)

Tensor:
array([[[1, 2],
        [3, 4]]], dtype=int32)

Tensor type:
<class 'mlx.core.array'>

Tensor dimension:
3

Tensor shape:
(1, 2, 2)



# Create a random tensor
MLX's random module provides flexible pseudo-random number generation (PRNG) with Apple Silicon optimization, balancing convenience and reproducibility through two key approaches

### Implicit Global State (Default):

In [61]:
import mlx.core.random as mxr

# Generates 3 unique values using global PRNG state
for _ in range(3):
    print(mxr.uniform())

array(0.451346, dtype=float32)
array(0.558482, dtype=float32)
array(0.258283, dtype=float32)


### explicit key control 

In [59]:
key = mxr.key(0)  # Seed initialization

# Generates identical values using fixed key
for _ in range(3):
    print(mxr.uniform(key=key))  # Output: 0.123, 0.123, 0.123

array(0.418457, dtype=float32)
array(0.418457, dtype=float32)
array(0.418457, dtype=float32)


In [64]:
def ml_rand(size, seed=None):
    """
    Generate an mlx.core.array filled with uniform random values.
    
    Parameters:
      size (tuple): The desired shape of the output array, e.g., (3, 4)
      seed (int, optional): Seed used to initialize the PRNG key for reproducibility.
                            If provided, each element is generated using a key based on (seed + index).
    
    Returns:
      An mlx.core.array with the specified shape filled with uniform random numbers.
    """
    import mlx.core.random as mxr
    import mlx.core as mx  # Assumes that mxa.array(...) creates an mlx array from a nested list

    # Compute the total number of elements.
    total_elements = 1
    for dim in size:
        total_elements *= dim

    # Generate flat list of random values.
    values = []
    for i in range(total_elements):
        if seed is not None:
            # Create a key based on the supplied seed plus an index offset.
            current_key = mxr.key(seed + i)
            num = mxr.uniform(key=current_key)
        else:
            # Use global PRNG state to generate a unique random value.
            num = mxr.uniform()
        values.append(num)
    
    # Helper: recursively reshape the flat list to the desired dimensions.
    def reshape(flat_list, shape):
        if len(shape) == 1:
            return flat_list[:shape[0]]
        else:
            sub_size = 1
            for d in shape[1:]:
                sub_size *= d
            return [reshape(flat_list[i * sub_size:(i + 1) * sub_size], shape[1:]) for i in range(shape[0])]
    
    nested_values = reshape(values, size)
    return mx.array(nested_values)

In [67]:
# Without a seed, the random values will be generated using the global PRNG state.
arr1 = ml_rand((3, 4))
print("Random array (global PRNG):")
print(arr1)

# With a seed, the values are reproducible.
arr2 = ml_rand((3, 4), seed=0)
print("\nRandom array (seed=0):")
print(arr2)

print_tensor_info(arr1)
print_tensor_info(arr2)

Random array (global PRNG):
array([[0.0685559, 0.749943, 0.808282, 0.383173],
       [0.108506, 0.28853, 0.499346, 0.710357],
       [0.835264, 0.810151, 0.112555, 0.554991]], dtype=float32)

Random array (seed=0):
array([[0.418457, 0.11815, 0.424022, 0.865393],
       [0.16215, 0.634341, 0.516013, 0.844107],
       [0.91191, 0.657114, 0.0893868, 0.452468]], dtype=float32)
Tensor:
array([[0.0685559, 0.749943, 0.808282, 0.383173],
       [0.108506, 0.28853, 0.499346, 0.710357],
       [0.835264, 0.810151, 0.112555, 0.554991]], dtype=float32)

Tensor type:
<class 'mlx.core.array'>

Tensor dimension:
2

Tensor shape:
(3, 4)

Tensor:
array([[0.418457, 0.11815, 0.424022, 0.865393],
       [0.16215, 0.634341, 0.516013, 0.844107],
       [0.91191, 0.657114, 0.0893868, 0.452468]], dtype=float32)

Tensor type:
<class 'mlx.core.array'>

Tensor dimension:
2

Tensor shape:
(3, 4)



# Device

In [32]:
# Get the default device.
mx.default_device()

Device(gpu, 0)